In [1]:
import os
import psycopg
from dotenv import load_dotenv



In [3]:
# Load environment variables from .env file
load_dotenv('../.env')
neon_conn_string = os.getenv("NEON_DB_URL")

In [5]:
try:
    with psycopg.connect(neon_conn_string) as neon_conn:
        print("Connection established")
        # Open a cursor to perform database operations
        with neon_conn.cursor() as cur:
            # Drop the table if it already exists
            cur.execute("DROP TABLE IF EXISTS books;")
            print("Finished dropping table (if it existed).")
            # Create a new table
            cur.execute("""
                CREATE TABLE books (
                    id SERIAL PRIMARY KEY,
                    title VARCHAR(255) NOT NULL,
                    author VARCHAR(255),
                    publication_year INT,
                    in_stock BOOLEAN DEFAULT TRUE
                );
            """)
            print("Finished creating table.")
            # Insert a single book record
            cur.execute(
                "INSERT INTO books (title, author, publication_year, in_stock) VALUES (%s, %s, %s, %s);",
                ("The Catcher in the Rye", "J.D. Salinger", 1951, True),
            )
            print("Inserted a single book.")
            # Data to be inserted
            books_to_insert = [
                ("The Hobbit", "J.R.R. Tolkien", 1937, True),
                ("1984", "George Orwell", 1949, True),
                ("Dune", "Frank Herbert", 1965, False),
            ]
            # Insert multiple books at once
            cur.executemany(
                "INSERT INTO books (title, author, publication_year, in_stock) VALUES (%s, %s, %s, %s);",
                books_to_insert,
            )
            print("Inserted 3 rows of data.")
            # The transaction is committed automatically when the 'with' block exits in psycopg (v3)
except Exception as e:
    print("Connection failed.")
    print(e)

Connection established
Finished dropping table (if it existed).
Finished creating table.
Inserted a single book.
Inserted 3 rows of data.


In [8]:
try:
    with psycopg.connect(neon_conn_string) as neon_conn:
        print("Connection established")
        # Open a cursor to perform database operations
        with neon_conn.cursor() as cur:
            cur.execute("SELECT * FROM books")
            print(cur.fetchall())
except Exception as e:
    print("Connection failed.")
    print(e)

Connection established
[(1, 'The Catcher in the Rye', 'J.D. Salinger', 1951, True), (2, 'The Hobbit', 'J.R.R. Tolkien', 1937, True), (3, '1984', 'George Orwell', 1949, True), (4, 'Dune', 'Frank Herbert', 1965, False)]


In [4]:
import pandas as pd

## Rooms table

First we need the rooms table as its primary key is referenced by the room_availability table.

In [5]:
df_rooms = pd.read_pickle('../data/pandas/rooms.pkl')
df_rooms

,room_id,room_number,floor,type,square_feet,basic_amenities,additional_amenities,max_occupancy,bed_type,view_type,accessibility,status,last_renovation,base_rate,max_rate
0,RM000001,101,1,Standard,410,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[City View],2,Queen,[City View],False,Available,2022-10-25,350,500
1,RM000002,102,1,Standard,361,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Evening Turndown Service],2,Queen,[Standard View],False,Available,2023-03-01,350,500
2,RM000003,103,1,Standard,431,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Courtyard View],2,Double Queen,[Courtyard View],False,Available,2022-08-10,350,500
3,RM000004,104,1,Standard,391,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[City View, Courtyard View, Evening Turndown S...",2,Queen,"[City View, Courtyard View]",True,Occupied,2023-09-06,350,500
4,RM000005,105,1,Standard,373,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[Courtyard View, City View, Evening Turndown S...",2,Double Queen,"[City View, Courtyard View]",True,Occupied,2024-04-13,350,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,RM000596,2026,20,Presidential Suite,2369,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Pool, Private Terrace, Luxury Car Ser...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2024-10-01,3500,5000
596,RM000597,2027,20,Presidential Suite,2020,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Grand Piano, Private Gym Equ...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2024-04-17,3500,5000
597,RM000598,2028,20,Presidential Suite,1883,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Grand Piano, Private Terrace, Dedicated Conci...",6,King + Multiple Sofa Beds,[Standard View],False,Occupied,2022-03-09,3500,5000
598,RM000599,2029,20,Presidential Suite,1643,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Private Chef Available, Priv...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2023-07-24,3500,5000


We want the basic_amenities, additional_amenities, and view_type to be saved as lists in table, so we need to modify them as such for PostgreSQL.

In [6]:
def to_sql_array_string(input_list):
    list2 = [x.replace('"', '\\"') for x in input_list]
    return '{"' + '", "'.join(list2) + '"}'

df_rooms['basic_amenities'] = df_rooms['basic_amenities'].apply(to_sql_array_string)
df_rooms['additional_amenities'] = df_rooms['additional_amenities'].apply(to_sql_array_string)
df_rooms['view_type'] = df_rooms['view_type'].apply(to_sql_array_string)

In [7]:
print(df_rooms.iloc[121])

room_id                                                          RM000122
room_number                                                           502
floor                                                                   5
type                                                               Deluxe
square_feet                                                           460
basic_amenities         {"Air Conditioning", "55\" Smart TV", "Nespres...
additional_amenities           {"Soaking Tub", "Pool View", "Ocean View"}
max_occupancy                                                           3
bed_type                                                     Double Queen
view_type                                     {"Pool View", "Ocean View"}
accessibility                                                       False
status                                                           Occupied
last_renovation                                                2024-11-25
base_rate                             

Convert the room_id to an int for more efficient use in SQL.

In [8]:
df_rooms['room_id'] = df_rooms['room_id'].apply(lambda x: int(x[2:]))
df_rooms

,room_id,room_number,floor,type,square_feet,basic_amenities,additional_amenities,max_occupancy,bed_type,view_type,accessibility,status,last_renovation,base_rate,max_rate
0,1,101,1,Standard,410,"{""Air Conditioning"", ""Smart TV"", ""Premium Coff...","{""City View""}",2,Queen,"{""City View""}",False,Available,2022-10-25,350,500
1,2,102,1,Standard,361,"{""Air Conditioning"", ""Smart TV"", ""Premium Coff...","{""Evening Turndown Service""}",2,Queen,"{""Standard View""}",False,Available,2023-03-01,350,500
2,3,103,1,Standard,431,"{""Air Conditioning"", ""Smart TV"", ""Premium Coff...","{""Courtyard View""}",2,Double Queen,"{""Courtyard View""}",False,Available,2022-08-10,350,500
3,4,104,1,Standard,391,"{""Air Conditioning"", ""Smart TV"", ""Premium Coff...","{""City View"", ""Courtyard View"", ""Evening Turnd...",2,Queen,"{""City View"", ""Courtyard View""}",True,Occupied,2023-09-06,350,500
4,5,105,1,Standard,373,"{""Air Conditioning"", ""Smart TV"", ""Premium Coff...","{""Courtyard View"", ""City View"", ""Evening Turnd...",2,Double Queen,"{""City View"", ""Courtyard View""}",True,Occupied,2024-04-13,350,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,2026,20,Presidential Suite,2369,"{""Air Conditioning"", ""Multiple 75\"" Smart TVs""...","{""Private Pool"", ""Private Terrace"", ""Luxury Ca...",6,King + Multiple Sofa Beds,"{""Standard View""}",False,Maintenance,2024-10-01,3500,5000
596,597,2027,20,Presidential Suite,2020,"{""Air Conditioning"", ""Multiple 75\"" Smart TVs""...","{""Private Terrace"", ""Grand Piano"", ""Private Gy...",6,King + Multiple Sofa Beds,"{""Standard View""}",False,Maintenance,2024-04-17,3500,5000
597,598,2028,20,Presidential Suite,1883,"{""Air Conditioning"", ""Multiple 75\"" Smart TVs""...","{""Grand Piano"", ""Private Terrace"", ""Dedicated ...",6,King + Multiple Sofa Beds,"{""Standard View""}",False,Occupied,2022-03-09,3500,5000
598,599,2029,20,Presidential Suite,1643,"{""Air Conditioning"", ""Multiple 75\"" Smart TVs""...","{""Private Terrace"", ""Private Chef Available"", ...",6,King + Multiple Sofa Beds,"{""Standard View""}",False,Maintenance,2023-07-24,3500,5000


Check that room_ids are sequential and start with 1

In [9]:
assert sum(df_rooms['room_id'].iloc[1:].to_numpy() - df_rooms['room_id'].iloc[:-1].to_numpy()) == df_rooms.shape[0]-1
assert df_rooms['room_id'].iloc[0] == 1

Add table to Neon

In [ ]:
try:
    with psycopg.connect(neon_conn_string) as neon_conn:
        print("Connection established")
        # Open a cursor to perform database operations
        with neon_conn.cursor() as cur:
            # Drop the table if it already exists
            cur.execute("DROP TABLE IF EXISTS room_availability;")
            print("Finished dropping table (if it existed).")
            # Create a new table
            cur.execute("""
                CREATE TABLE rooms (
                    room_id INT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
                    room_number INT NOT NULL,
                    floor INT NOT NULL,
                    type VARCHAR(50),
                    square_feet INT,
                    basic_amenities TEXT[],  -- Array data type for basic amenities
                    additional_amenities TEXT[], -- Array data type for additional amenities
                    max_occupancy INT,
                    bed_type VARCHAR(50),
                    view_type TEXT[],  -- Array data type for view type
                    accessibility BOOLEAN,
                    status VARCHAR(50),
                    last_renovation DATE,
                    base_rate NUMERIC(10, 2),
                    max_rate NUMERIC(10, 2)
                );
            """)
            print("Finished creating table.")
            # Insert room availability data
            cur.executemany(
                "INSERT INTO room_availability (room_id, room_number, date, status, price, max_occupancy) VALUES (%s, %s, %s, %s, %s, %s);",
                df_room_availability.values.tolist(),
            )
            print("Inserted data.")
            # The transaction is committed automatically when the 'with' block exits in psycopg (v3)
except Exception as e:
    print("Connection failed.")
    print(e)

## Room Availability table

Now add the room_availability table

In [10]:
df_room_availability = pd.read_pickle('../data/pandas/room_availability.pkl')
df_room_availability

,room_id,room_number,date,status,price,max_occupancy
0,RM000001,101,2025-01-04,Booked,NaN,2
1,RM000001,101,2025-01-05,Booked,NaN,2
2,RM000001,101,2025-01-06,Booked,NaN,2
3,RM000001,101,2025-01-07,Booked,NaN,2
4,RM000001,101,2025-01-08,Booked,NaN,2
...,...,...,...,...,...,...
218995,RM000600,2030,2025-12-30,Booked,NaN,6
218996,RM000600,2030,2025-12-31,Booked,NaN,6
218997,RM000600,2030,2026-01-01,Booked,NaN,6
218998,RM000600,2030,2026-01-02,Available,4617.6,6


Convert room_id to int for more efficient use in SQL and because used that way in rooms table.

In [11]:
df_room_availability['room_id'] = df_room_availability['room_id'].apply(lambda x: int(x[2:]))
df_room_availability

,room_id,room_number,date,status,price,max_occupancy
0,1,101,2025-01-04,Booked,NaN,2
1,1,101,2025-01-05,Booked,NaN,2
2,1,101,2025-01-06,Booked,NaN,2
3,1,101,2025-01-07,Booked,NaN,2
4,1,101,2025-01-08,Booked,NaN,2
...,...,...,...,...,...,...
218995,600,2030,2025-12-30,Booked,NaN,6
218996,600,2030,2025-12-31,Booked,NaN,6
218997,600,2030,2026-01-01,Booked,NaN,6
218998,600,2030,2026-01-02,Available,4617.6,6


In [12]:
df_room_availability.values.tolist()

[[1, 101, '2025-01-04', 'Booked', nan, 2],
 [1, 101, '2025-01-05', 'Booked', nan, 2],
 [1, 101, '2025-01-06', 'Booked', nan, 2],
 [1, 101, '2025-01-07', 'Booked', nan, 2],
 [1, 101, '2025-01-08', 'Booked', nan, 2],
 [1, 101, '2025-01-09', 'Booked', nan, 2],
 [1, 101, '2025-01-10', 'Booked', nan, 2],
 [1, 101, '2025-01-11', 'Booked', nan, 2],
 [1, 101, '2025-01-12', 'Booked', nan, 2],
 [1, 101, '2025-01-13', 'Booked', nan, 2],
 [1, 101, '2025-01-14', 'Booked', nan, 2],
 [1, 101, '2025-01-15', 'Booked', nan, 2],
 [1, 101, '2025-01-16', 'Booked', nan, 2],
 [1, 101, '2025-01-17', 'Booked', nan, 2],
 [1, 101, '2025-01-18', 'Booked', nan, 2],
 [1, 101, '2025-01-19', 'Booked', nan, 2],
 [1, 101, '2025-01-20', 'Available', 500.0, 2],
 [1, 101, '2025-01-21', 'Available', 500.0, 2],
 [1, 101, '2025-01-22', 'Available', 468.57, 2],
 [1, 101, '2025-01-23', 'Available', 485.14, 2],
 [1, 101, '2025-01-24', 'Available', 500.0, 2],
 [1, 101, '2025-01-25', 'Available', 500.0, 2],
 [1, 101, '2025-01-26'

In [ ]:
try:
    with psycopg.connect(neon_conn_string) as neon_conn:
        print("Connection established")
        # Open a cursor to perform database operations
        with neon_conn.cursor() as cur:
            # Drop the table if it already exists
            cur.execute("DROP TABLE IF EXISTS room_availability;")
            print("Finished dropping table (if it existed).")
            # Create a new table
            cur.execute("""
                CREATE TABLE room_availability (
                    id GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
                    room_id INT NOT NULL,
                    room_number INT NOT NULL,
                    date DATE NOT NULL,
                    status VARCHAR(15) NOT NULL,
                    price NUMERIC(8,2),
                    max_occupancy INT,
                    FOREIGN KEY (room_id) REFERENCES rooms(room_id)
                );
            """)
            print("Finished creating table.")
            # Insert room availability data
            cur.executemany(
                "INSERT INTO room_availability (room_id, room_number, date, status, price, max_occupancy) VALUES (%s, %s, %s, %s, %s, %s);",
                df_room_availability.values.tolist(),
            )
            print("Inserted data.")
            # The transaction is committed automatically when the 'with' block exits in psycopg (v3)
except Exception as e:
    print("Connection failed.")
    print(e)

Connection established
Finished dropping table (if it existed).
Finished creating table.
Inserted data.


In [ ]:
try:
    with psycopg.connect(neon_conn_string) as neon_conn:
        print("Connection established")
        # Open a cursor to perform database operations
        with neon_conn.cursor() as cur:
            # Drop the table if it already exists
            cur.execute("SELECT COUNT(*) FROM room_availability")
            print(cur.fetchall())
except Exception as e:
    print("Connection failed.")
    print(e)

Connection established
[(219000,)]


In [51]:
from openai import OpenAI

class NL2SQLAgent:
    def __init__(self, db_conn, llm_conn):
        self.db = db_conn
        self.llm = llm_conn

    def handle_nl_query(self, query: str):
        instructions = """
            You are a SQL query writer. Given the user's input, write a query that, when
            passed to a PostgreSQL database, will gather the data necessary to answer
            the user's query.
            
            You have access to the table room_availability, which provides information
            on the availability of rooms in a hotel, with the following columns:
             - room_number: The room number
             - date: The date the availability information in this row is for
             - status: Whether the room is available or not. Options are Available, Booked, and Maintenance.
             - price: The current price to book the room for that date
             - max_occupancy: The maximum number of people who can stay in the room
        """
        
        return self.llm.responses.create(
            model="gpt-5-mini",
            instructions=instructions,
            input=query,
        )
    
neon_conn = psycopg.connect(neon_conn_string).cursor()
openai_conn = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
nl_2_sql_agent = NL2SQLAgent(neon_conn, openai_conn)

In [36]:
def parse_to_sql(nl_query: str) -> str:
    return nl_2_sql_agent.handle_nl_query(nl_query)

In [ ]:
def test_nl2sql():
    query = "Show me available rooms under $500"
    sql = parse_to_sql(query)
    return sql
    
response_query = test_nl2sql()

In [ ]:
neon_conn.execute(response_query.output_text).fetchall()

[(419, datetime.date(2025, 11, 27), Decimal('315.07'), 2),
 (219, datetime.date(2025, 11, 18), Decimal('315.15'), 2),
 (409, datetime.date(2025, 11, 13), Decimal('315.27'), 2),
 (307, datetime.date(2025, 11, 28), Decimal('315.45'), 2),
 (212, datetime.date(2025, 11, 25), Decimal('315.46'), 2),
 (409, datetime.date(2025, 11, 17), Decimal('315.46'), 2),
 (322, datetime.date(2025, 11, 12), Decimal('315.50'), 2),
 (301, datetime.date(2025, 11, 12), Decimal('315.56'), 2),
 (113, datetime.date(2025, 11, 25), Decimal('315.58'), 2),
 (324, datetime.date(2025, 11, 28), Decimal('315.63'), 2),
 (104, datetime.date(2025, 11, 13), Decimal('315.67'), 2),
 (104, datetime.date(2025, 11, 20), Decimal('315.69'), 2),
 (322, datetime.date(2025, 11, 24), Decimal('315.73'), 2),
 (127, datetime.date(2025, 11, 21), Decimal('316.01'), 2),
 (413, datetime.date(2025, 11, 17), Decimal('316.06'), 2),
 (107, datetime.date(2025, 11, 20), Decimal('316.12'), 2),
 (320, datetime.date(2025, 11, 13), Decimal('316.12'), 2

,room_number,floor,type,square_feet,basic_amenities,additional_amenities,max_occupancy,bed_type,view_type,accessibility,status,last_renovation,base_rate,max_rate
room_id,,,,,,,,,,,,,,
RM000001,101,1,Standard,410,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[City View],2,Queen,City View,False,Available,2022-10-25,350,500
RM000002,102,1,Standard,361,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Evening Turndown Service],2,Queen,Standard View,False,Available,2023-03-01,350,500
RM000003,103,1,Standard,431,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Courtyard View],2,Double Queen,Courtyard View,False,Available,2022-08-10,350,500
RM000004,104,1,Standard,391,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[City View, Courtyard View, Evening Turndown S...",2,Queen,Courtyard View,True,Occupied,2023-09-06,350,500
RM000005,105,1,Standard,373,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[Courtyard View, City View, Evening Turndown S...",2,Double Queen,Evening Turndown Service,True,Occupied,2024-04-13,350,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RM000596,2026,20,Presidential Suite,2369,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Pool, Private Terrace, Luxury Car Ser...",6,King + Multiple Sofa Beds,Standard View,False,Maintenance,2024-10-01,3500,5000
RM000597,2027,20,Presidential Suite,2020,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Grand Piano, Private Gym Equ...",6,King + Multiple Sofa Beds,Standard View,False,Maintenance,2024-04-17,3500,5000
RM000598,2028,20,Presidential Suite,1883,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Grand Piano, Private Terrace, Dedicated Conci...",6,King + Multiple Sofa Beds,Standard View,False,Occupied,2022-03-09,3500,5000
